In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from collections import defaultdict
import cv2
import gc
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torchvision.models import resnet18

from config import *
from htools import hdir
from models import (conv_block, ResBlock, GRelu, JRelu, BaseModel,
                    CycleGenerator, Discriminator)
from torch_datasets import sketch_dl, photo_dl
from utils import render_samples, show_img, show_samples

In [6]:
# # NO NEED TO RUN, IMPORTED FROM CONFIG - JUST FOR EASY REFERENCE

# bs = 64                # Batch size (paper uses 128).
# img_size = 64          # Size of input (here it's 64 x 64).
# workers = 2            # Number of workers for data loader.
# input_c = 100          # Depth of input noise (1 x 1 x noise_dim). AKA nz.
# ngf = 64               # Filters in first G layer.
# ndf = 64               # Filters in first D layer.
# lr = 2e-4              # Recommended learning rate of .0002.
# beta1 = .5             # Recommended parameter for Adam.
# nc = 3                 # Number of channels of input image.
# ngpu = 1               # Number of GPUs to use.
# sample_dir = 'samples' # Directory to store sample images from G. 
# weight_dir = 'weights' # Directory to store model weights.
# device = torch.device('cuda:0' if torch.cuda.is_available() and ngpu > 0 
#                       else 'cpu')

In [4]:
# class ResBlock(nn.Module):
#     """Residual block to be used in CycleGenerator. Note that the relu or 
#     leaky must still be applied on the output.
#     """
    
#     def __init__(self, c_in, activation=GRelu(.02), num_layers=2, norm='bn'):
#         """
#         Parameters
#         -----------
#         c_in: int
#             # of input channels.
#         num_layers: int
#             Number of conv blocks inside the skip connection (default 2). 
#             ResNet paper notes that skipping a single layer did not show
#             noticeable improvements.
#         leak: float
#             Slope of leaky relu where x < 0.
#         norm: str
#             'bn' for batch norm, 'in' for instance norm
#         """
#         super().__init__()
#         self.layers = nn.ModuleList([conv_block(False, c_in, c_in, 3, 1, 1, norm=norm) 
#                                      for i in range(num_layers)])
#         self.activation = activation
    
#     def forward(self, x):
#         x_out = x
#         for layer in self.layers:
#             x_out = self.activation(layer(x_out))
#         return x + x_out

In [192]:
# class CycleGenerator(BaseModel):
#     """CycleGAN Generator."""

#     def __init__(self, img_c=3, ngf=64, norm='bn', act=GRelu(.02)):
#         """
#         Parameters
#         -----------
#         img_c: int
#             # of channels of input image.
#         ngf: int
#             # of channels in first convolutional layer.
#         norm: str
#             Type of normalization layer used for res blocks in the 
#             transformer. Default is 'bn' for batch norm, but can also use
#             'in' for instance norm.
#         act: nn.Module
#             Default activation of GRelu(.02) gives us a leaky relu with a leak
#             of .02, as recommended in the paper. JRelu (already instantiated 
#             so exclude parentheses) gives a leak of .1, sub of .4, and max of 
#             6.0. GRelu() gives a standard ReLU.
#         """
#         super().__init__()
#         self.activation = act

#         # ENCODER
#         # 3 x 64 x 64 -> 64 x 32 x 32
#         deconv1 = conv_block(True, img_c, ngf, f=4, stride=2, pad=1)
#         # 64 x 32 x 32 -> 128 x 16 x 16
#         deconv2 = conv_block(True, ngf, ngf*2, 4, 2, 1)
#         self.encoder = nn.Sequential(deconv1, 
#                                      self.activation,
#                                      deconv2,
#                                      self.activation)

#         # TRANSFORMER
#         # 128 x 16 x 16 -> 128 x 16 x 16
#         res1 = ResBlock(ngf*2, self.activation, num_layers=2, norm=norm)
#         # 128 x 16 x 16 -> 128 x 16 x 16
#         res2 = ResBlock(ngf*2, self.activation, 2, norm)
#         self.transformer = nn.Sequential(res1,
#                                          self.activation,
#                                          res2,
#                                          self.activation)

#         # DECODER
#         # 128 x 16 x 16 -> 64 x 32 x 32
#         deconv1 = conv_block(False, ngf*2, ngf, f=4, stride=2, pad=1)
#         # 64 x 32 x 32 -> 3 x 64 x 64
#         deconv2 = conv_block(False, ngf, img_c, 4, 2, 1)
#         self.decoder = nn.Sequential(deconv1, 
#                                      self.activation,
#                                      deconv2,
#                                      nn.Tanh())

#         # Module list of Sequential objects is helpful if we want to use 
#         # different learning rates per group.
#         self.groups = nn.ModuleList([self.encoder,
#                                      self.transformer,
#                                      self.decoder])

#     def forward(self, x):
#         for group in self.groups:
#             x = group(x)
#         return x

## IDEA: maybe we can use 1 discriminator with output classes real x, real y, fake x, fake y?

In [6]:
class PretrainedDiscriminator(BaseModel):
    
    def __init__(self):
        super().__init__()
        layers = list(resnet18(pretrained=True).children())[:-2]
#       # WAS THINKING OF W/ REPLACING RELU W/ GRELU; BIT TRICKY W/ SEQUENTIAL
#         self.act = act
#         for layer in layers:
#             if layer.__class__.__name__ == 'ReLU':
#                 layer = self.act
        self.groups = nn.ModuleList([nn.Sequential(*layers[:6]),
                                     nn.Sequential(*layers[6:])])
        self.freeze()
        self.groups.append(nn.Linear(512, 1))
        
    def forward(self, x):
        for group in self.groups[:-1]:
            x = group(x)
        x = F.relu(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = self.groups[-1](x.view(x.shape[0], -1))
        return torch.sigmoid(x).squeeze()
    
    def freeze(self):
        """Freeze the whole network."""
        for group in self.groups:
            for p in group.parameters():
                p.requires_grad = False
                
    def unfreeze(self, i=None, verbose=False):
        """Unfreeze all or part of the network.
        
        Parameters
        -----------
        i: None or int
            If None, unfreeze the whole network. If int, unfreeze only that
            group.
        """
        for j, group in enumerate(self.groups):
            if i is None or j == i:
                if verbose: print(f'Unfreezing group {j}.')
                for p in group.parameters():
                    p.requires_grad = True
            else:
                if verbose: print(f'Freezing group {j}.')
                for p in group.parameters():
                    p.requires_grad = False

In [221]:
x, y = next(iter(photo_dl))

In [222]:
G = CycleGenerator()
y_fake = G(x)
y_fake.shape

torch.Size([64, 3, 64, 64])

In [233]:
D = PretrainedDiscriminator()
pred_y_fake = D(y_fake)

## Check - photo and sketch dataloaders may not always have same batch size?
-also check: can we use class labels, not just 1's and 0's?

In [7]:
def train_cycle_gan(epochs, x_dl, y_dl, sample_dir_x, sample_dir_y, 
                    weight_dir=None, sample_freq=10, lr=2e-4, b1=.5,
                    use_labels=False, quiet_mode=True, load_path=None,
                    models=None):
    """Train cycleGAN with Adam optimizer. The naming conventin G_xy will be
    used to refer to a generator that converts from set x to set y, while
    D_x refers to a discriminator that classifies examples as actually 
    belonging to set x (class 1) or being a model-generated example (class 0).
    
    Parameters
    -----------
    use_labels: bool
        Specifies whether to use class labels (i.e. horse, zebra, giraffe). If
        False, D only tries to predict if it is a real or fake example 
        (e.g. photo or sketch2photo).
    load_path: str or None
        If str, models will load state dicts from the provided file. If None,
        new models will be created.
    models: list or None
        Instead of providing a load path, we can also pass in a list of models
        in the form [G_xy, G_yx, D_x, D_y]. Either load_path or models 
        (or both) should be None.
    """
    # Create models.
    if not models:
        G_xy, G_yx, D_x, D_y = get_cycle_models(load_path)
    else:
        for model in models:
            model.to(device)
            model.train()
        G_xy, G_yx, D_x, D_y = models
    
    # Create optimizers.
    optim_g = torch.optim.Adam(chain(G_xy.parameters(), G_yx.parameters()), 
                               lr, betas=(b1, .999))
    optim_d = torch.optim.Adam(chain(D_x.parameters(), D_y.parameters()),
                               lr, betas=(b1, .999))    
    
    # Define loss function.
    if use_labels:
        criterion = nn.BCELoss(reduction='mean')
    else:
        criterion = nn.MSELoss(reduction='mean')
        
    # Set fixed examples for sample generation.
    fixed_x = next(iter(x_dl))[0]
    fixed_y = next(iter(y_dl))[0]
    
    # Suppress printed output to 20 total times to avoid slowing down nb.
    print_freq = 1
    if quiet_mode:
        print_freq = max(1, epochs // 20)
    
    # Store lists of losses.
    stats = defaultdict(list)
    for epoch in range(epochs):
        G_xy.train()
        G_yx.train()
        D_x.train()
        D_y.train()
        
        for i, ((x, x_labels), (y, y_labels)) in enumerate(zip(x_dl, y_dl)):
            x = x.to(device)
            y = y.to(device)
            batch_len = x.shape[0]
            # X has less data so DL runs out and last batch is small. FIX?
            if batch_len != y.shape[0]:
                continue
            labels_real = torch.ones(batch_len, device=device)
            labels_fake = torch.zeros(batch_len, device=device)
            
            ##################################################################
            # Train D_x and D_y.
            ##################################################################
            # Train D's on real images.
            optim_d.zero_grad()
            pred_x, pred_y = D_x(x), D_y(y)
            loss_dx = criterion(pred_x, labels_real)
            loss_dy = criterion(pred_y, labels_real)
            loss_d_real = loss_dx + loss_dy
            stats['d_real'].append(loss_d_real.item())
            loss_d_real.backward()
            optim_d.step()
            
            # Train D's on fake images.
            optim_d.zero_grad()
            x_fake, y_fake = G_yx(y), G_xy(x)
            pred_x, pred_y = D_x(x_fake), D_y(y_fake)
            loss_dx_fake = criterion(pred_x, labels_fake)
            loss_dy_fake = criterion(pred_y, labels_fake)
            loss_d_fake = loss_dx_fake + loss_dy_fake
            stats['d_fake'].append(loss_d_fake.item())
            loss_d_fake.backward()
            optim_d.step()
            
            ##################################################################
            # Train G_xy and G_yx.
            ################################################################## 
            # Stage 1: x -> y -> x
            optim_g.zero_grad()
            y_fake = G_xy(x)
            pred_y = D_y(y_fake)
            loss_g = criterion(pred_y, labels_real)
            
            x_recon = G_yx(y_fake)
            pred_x = D_x(x_recon)
            loss_g_cycle = criterion(pred_x, labels_real)
            loss_g_total = loss_g + loss_g_cycle
            stats['g_xyx'].append(loss_g_total.item())
            loss_g_total.backward()
            optim_g.step()
            
            # Stage2: y -> x -> y
            optim_g.zero_grad()
            x_fake = G_yx(y)
            pred_x = D_x(x_fake)
            loss_g = criterion(pred_x, labels_real)
            
            y_recon = G_xy(x_fake)
            pred_y = D_y(y_recon)
            loss_g_cycle = criterion(pred_y, labels_real)
            loss_g_total = loss_g + loss_g_cycle
            stats['g_yxy'].append(loss_g_total.item())
            loss_g_total.backward()
            optim_g.step()
        
        # Generate samples to save at specified intervals.
        if epoch % sample_freq == 0:
            sample_x = G_yx(y).detach()
            sample_y = G_yx(x).detach()
            vutils.save_image(sample_x, f'{sample_dir_x}/{epoch}.png')
            vutils.save_image(sample_y, f'{sample_dir_y}/{epoch}.png')
            
       # If specified, save weights corresponding to generated samples.
        if weight_dir:
            states = dict(g_xy=G_xy.state_dict(),
                          g_yx=G_yx.state_dict(),
                          dx=D_x.state_dict(),
                          dy=D_y.state_dict(),
                          epoch=epoch)
            torch.save(states, f'{weight_dir}/{epoch}.pth') 
                
        # Print results for last mini batch of epoch.
        if epoch % print_freq == 0:
            print(f'Epoch [{epoch+1}/{epochs}])')
            print(f"D real loss: {loss_d_real:.3f}\t"
                  f"D fake loss: {loss_d_fake:.3f}")
            print(f"G xyx loss: {stats['g_xyx'][-1]:.3f}\t"
                  f"G yxy fake: {stats['g_yxy'][-1]:.3f}\n")
            
    return stats

In [8]:
def get_cycle_models(path=None):
    """Get 2 cycle generators and 2 discriminators. If a path to a weight file
    is provided, load state dicts from that file. Models are returned in train
    mode.
    
    Parameters
    -----------
    path: str
        Optional - pass in path to weights file to load previously saved state
        dicts, or exclude to get new models.
    """
    models = dict(g_xy=CycleGenerator(), g_yx=CycleGenerator(), 
                  d_x=Discriminator(), d_y=Discriminator())
    if path:
        states = torch.load(path)
        print(f"Loading models from epoch {states['epoch']}.")
    for name, model in models.items():
        if path:
            model.load_state_dict(states[name])
        model.to(device)
        model.train()
    print('All models currently in training mode.')
    return list(models.values())

In [9]:
m = get_cycle_models()

All models currently in training mode.


In [15]:
stats = train_cycle_gan(5, photo_dl, sketch_dl, 'samples_cycle_photo', 
                        'samples_cycle_sketch', sample_freq=1)

All models currently in training mode.
Epoch [1/5])
D real loss: 0.000	D fake loss: 0.000
G xyx loss: 1.000	G yxy fake: 1.000

Epoch [2/5])
D real loss: 0.000	D fake loss: 0.000
G xyx loss: 1.000	G yxy fake: 1.000

Epoch [3/5])
D real loss: 0.000	D fake loss: 0.000
G xyx loss: 1.000	G yxy fake: 1.000

Epoch [4/5])
D real loss: 0.000	D fake loss: 0.000
G xyx loss: 1.000	G yxy fake: 1.000

Epoch [5/5])
D real loss: 0.000	D fake loss: 0.000
G xyx loss: 1.000	G yxy fake: 1.000



In [265]:
G_xy = CycleGenerator(img_c, ngf)
G_yx = CycleGenerator()

In [259]:
D = Discriminator(ndf)

In [261]:
x.shape

torch.Size([2, 3, 4, 4])

In [272]:
y_hat = G_xy(x)
y_hat.shape

torch.Size([2, 3, 4, 4])

In [271]:
G_yx(y_hat).shape

torch.Size([2, 3, 4, 4])